In [1]:
#Functions with descriptions in portuguese for algorithm and results. Implementation starts in step 2

def algoritgmDescription():
    print("")
    print("Cada pacote gerado no tracer contém um tempo de envio e um tempo de recebimento. A partir desses dois valores" +
         " são feitos calculos para gerar a perda real e estimada e a variação real e estimada, para que de acordo com os " +
         "resultados seja gerado o tempo em que o pacote será reproduzido, a fim de prever se o mesmo será perdido ou não, onde um pacote é dado como perdido " +
          "quando o seu tempo de reprodução é maior que o tempo de recebimento do pacote." +
         " O objetivo do algoritmo é reduzir o atraso na reprodução de pacotes ajustando os atrasos de reprodução adaptativamente.")
    
    print("")
    print("Segue abaixo a implementação do algoritimo (Atraso por reprodução adaptativa) contendo as regras descritas de acordo com o livro [Kurose - Ross - 6ª edição]")
    print("")

def rateDescription(rate):
    
    conclusion = ""
    if rate < 15.00:
        conclusion = "Sendo assim hábil de realizar a técnica de ocutação de erro!"
    else:
        conclusion = "Não sendo hábil de realizar a técnica de ocutação de erro!"
    
    print("")
    print("A técnica utilizada para ocutação de erro, descrita no livro [Kurose - Ross - 6ª edição],",
      "faz uma recuperação dos pacotes perdidos baseando-se no receptor, substituindo por um pacote semelhante,",
     "necessitando alta semelhança entre si. O autor diz que essa técnica é eficiente para taxas de perda baixas, menores que 15%.")
    print("")
    print("Visualizando a taxa de perda de pacotes, analisando os traces gerados, temos um valor de " + str(rate) + "%. " +
     conclusion)
    print("")

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("traces.csv",sep = ";")

In [4]:
# Constants
U = 0.001998
U2 = (1 - U)
K = 4

# Variables
lastLostEstimated = 1376
lastEstimateVariate = 0.00
lastType = '!'
countLosted = 0
countDontLosted = 0

# Arrays
receiver_times = []
reproduction_time = []
status = []

In [5]:
algoritgmDescription()


Cada pacote gerado no tracer contém um tempo de envio e um tempo de recebimento. A partir desses dois valores são feitos calculos para gerar a perda real e estimada e a variação real e estimada, para que de acordo com os resultados seja gerado o tempo em que o pacote será reproduzido, a fim de prever se o mesmo será perdido ou não, onde um pacote é dado como perdido quando o seu tempo de reprodução é maior que o tempo de recebimento do pacote. O objetivo do algoritmo é reduzir o atraso na reprodução de pacotes ajustando os atrasos de reprodução adaptativamente.

Segue abaixo a implementação do algoritimo (Atraso por reprodução adaptativa) contendo as regras descritas de acordo com o livro [Kurose - Ross - 6ª edição]



In [6]:
for idx, list_traces in data.iterrows():
    if list_traces['type'] == 'D':
        
        senderTimestapm = list_traces['sender_timestamp']
        receiverTimestamp = list_traces['receiver_timestamp']          
        
        realLost = receiverTimestamp - senderTimestapm        

        lostEstimated = ((U2 * lastLostEstimated) + (U * realLost))    
        lostEstimated = round(lostEstimated, 6)
        lastLostEstimated = lostEstimated    

        variate = abs(realLost - lostEstimated)
        variate = round(variate, 6)
        
        estimateVariate = ((U2 * lastEstimateVariate) + (U * variate))    
        estimateVariate = round(estimateVariate, 6)
        lastEstimateVariate = estimateVariate
  
        if lastType == '!':
            lastType = 'D'
            pi = (senderTimestapm + lostEstimated + (K * estimateVariate))
            pi = round(pi, 0)
            lastPi = pi
        else:
            pi = lastPi + 160
            pi = round(pi, 0)
            lastPi = pi
        if lastPi < receiverTimestamp:
            countLosted = countLosted + 1;                        
            losted = 1
            
            receiver_times.append(receiverTimestamp)
            reproduction_time.append(lastPi)
            status.append('LOSTED')
        else:            
            countDontLosted = countDontLosted + 1;
            losted = 0        
    else:        
        lastType = '!'
        continue

In [7]:
print("")
print("Pacotes Ok: ", countDontLosted)
print("Pacotes Perdidos: ", countLosted)
total = countDontLosted + countLosted
print("Total: ", total)
lostPacketRate = countLosted / total
rate = round(lostPacketRate * 100, 2)
print("")


Pacotes Ok:  35063
Pacotes Perdidos:  2577
Total:  37640



In [8]:
rateDescription(rate)


A técnica utilizada para ocutação de erro, descrita no livro [Kurose - Ross - 6ª edição], faz uma recuperação dos pacotes perdidos baseando-se no receptor, substituindo por um pacote semelhante, necessitando alta semelhança entre si. O autor diz que essa técnica é eficiente para taxas de perda baixas, menores que 15%.

Visualizando a taxa de perda de pacotes, analisando os traces gerados, temos um valor de 6.85%. Sendo assim hábil de realizar a técnica de ocutação de erro!



In [10]:
#Generated csv file with results for lost packets.

data = {
    'receiver_time': receiver_times,
    'reproduction_time': reproduction_time,
    'status': status
}

results = pd.DataFrame(data)

results.to_csv('results.csv')

dataResults = pd.read_csv("results.csv",sep = ";")